In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import glob
import os
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality plot parameters
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
    'legend.fontsize': 11,
    'figure.titlesize': 16,
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif', 'serif'],
    'axes.linewidth': 1.2,
    'grid.linewidth': 0.8,
    'lines.linewidth': 2,
    'patch.linewidth': 0.5,
    'xtick.major.width': 1.2,
    'ytick.major.width': 1.2,
    'xtick.minor.width': 0.8,
    'ytick.minor.width': 0.8,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    #'axes.grid.alpha': 0.3,
    'grid.alpha': 0.3
})

# Define a professional color palette
COLORS = {
    'primary': '#2E86AB',      # Professional blue
    'secondary': '#A23B72',    # Deep magenta
    'accent': '#F18F01',       # Warm orange
    'success': '#C73E1D',      # Deep red
    'neutral': '#6C757D',      # Gray
    'light': '#E9ECEF',        # Light gray
    'multimodal': '#FF6B6B',   # Coral red
    'unimodal': '#4ECDC4',     # Teal
    'gradient': ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6C757D']
}

def load_all_evaluation_data(results_folder):
    """Load all JSON evaluation results from folder"""
    json_files = glob.glob(os.path.join(results_folder, "*.json"))
    
    print(f"Found {len(json_files)} result files in {results_folder}")
    
    all_data = []
    run_summaries = []
    
    for file_path in sorted(json_files):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            # Extract run summary
            run_summary = {
                'run_id': data['run_id'],
                'total_questions': data['total_questions'],
                'correct_answers': data['correct_answers'],
                'accuracy': data['accuracy'],
                'duration': data['duration'],
                'avg_time_per_question': data['avg_time_per_question'],
                'timestamp': data['timestamp'],
                'file_path': file_path
            }
            run_summaries.append(run_summary)
            
            # Add run metadata to each result
            for result in data['results']:
                result['run_timestamp'] = data['timestamp']
                result['run_duration'] = data['duration']
                result['run_accuracy'] = data['accuracy']
                
            all_data.extend(data['results'])
            
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    return all_data, run_summaries

def load_original_dataset(dataset_path):
    """Load the original JEE dataset to get requires_image field"""
    try:
        df_original = pd.read_csv(dataset_path)
        # Create a mapping from question_id to requires_image
        image_mapping = dict(zip(df_original['question_id'], df_original['requires_image']))
        return image_mapping
    except Exception as e:
        print(f"Error loading original dataset {dataset_path}: {e}")
        return {}

def load_multi_model_data(model_folders, dataset_path):
    """Load data from multiple model folders"""
    all_models_data = {}
    image_mapping = load_original_dataset(dataset_path)
    
    for model_name, folder_path in model_folders.items():
        print(f"\nLoading data for {model_name}...")
        all_data, run_summaries = load_all_evaluation_data(folder_path)
        
        if all_data:
            df = pd.DataFrame(all_data)
            
            # Add multimodal information from original dataset
            df['requires_image'] = df['question_id'].map(image_mapping)
            df['is_multimodal'] = df['requires_image'].fillna(False)
            
            # Add model name
            df['model'] = model_name
            
            all_models_data[model_name] = {
                'df': df,
                'run_summaries': pd.DataFrame(run_summaries)
            }
            
            print(f"Loaded {len(df)} questions for {model_name}")
        else:
            print(f"No data found for {model_name}")
    
    return all_models_data

def create_accuracy_heatmap_by_model(all_models_data, save_path=None):
    """Create publication-quality accuracy heatmap for language performance only"""
    
    # Combine all data
    combined_df = pd.concat([data['df'] for data in all_models_data.values()], ignore_index=True)
    
    # Create pivot table for model vs language accuracy
    model_language_accuracy = combined_df.groupby(['model', 'language'])['is_correct'].mean().reset_index()
    pivot_model_language = model_language_accuracy.pivot(index='model', columns='language', values='is_correct')
    
    # Create the plot with larger fonts for publication quality
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Custom colormap for better contrast
    cmap = sns.diverging_palette(250, 10, n=256, as_cmap=True)
    
    # Model vs Language heatmap with larger fonts
    im = sns.heatmap(pivot_model_language, 
                     annot=True, 
                     fmt='.3f',  # Show 3 decimal places for precision
                     cmap=cmap,
                     center=pivot_model_language.values.mean(),
                     square=False,
                     linewidths=1.0,  # Slightly thicker lines
                     linecolor='white',
                     cbar_kws={
                         'label': 'Accuracy',
                         'shrink': 0.8,
                         'aspect': 15,
                         'pad': 0.02
                     },
                     annot_kws={'fontsize': 16, 'fontweight': 'bold'},  # Larger annotation font
                     ax=ax)
    
    # Remove title as requested
    # ax.set_title('Model Performance by Language', fontweight='bold', pad=20)
    
    # Larger axis labels
    ax.set_xlabel('Language', fontweight='bold', fontsize=18)
    ax.set_ylabel('Model', fontweight='bold', fontsize=18)
    
    # Larger tick labels
    ax.tick_params(axis='x', rotation=45, labelsize=16)
    ax.tick_params(axis='y', rotation=0, labelsize=16)
    
    # Larger colorbar label
    cbar = ax.collections[0].colorbar
    cbar.set_label('Accuracy', fontweight='bold', fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    
    # Fine-tune layout
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, format='pdf', bbox_inches='tight', dpi=300, 
                   facecolor='white', edgecolor='none')
        print(f"✓ Language accuracy heatmap saved to {save_path}")
    
    plt.show()
    
    return pivot_model_language

def create_multimodal_comparison(all_models_data, save_path=None):
    """Create publication-quality multimodal vs unimodal performance comparison - Bar chart only"""
    
    # Combine all data
    combined_df = pd.concat([data['df'] for data in all_models_data.values()], ignore_index=True)
    
    # Filter out rows where requires_image is NaN
    combined_df = combined_df.dropna(subset=['requires_image'])
    
    # Calculate performance by model and modality
    multimodal_performance = combined_df.groupby(['model', 'is_multimodal'])['is_correct'].mean().reset_index()
    
    # Create pivot for plotting
    pivot_multimodal = multimodal_performance.pivot(index='model', columns='is_multimodal', values='is_correct')
    pivot_multimodal.columns = ['Text-Only', 'Multimodal']
    
    # Calculate sample sizes for annotations
    sample_sizes = combined_df.groupby(['model', 'is_multimodal']).size().reset_index(name='count')
    sample_pivot = sample_sizes.pivot(index='model', columns='is_multimodal', values='count')
    sample_pivot.columns = ['Text-Only', 'Multimodal']
    
    # Create the publication-quality plot - Single bar chart only
    fig, ax = plt.subplots(figsize=(12, 8))
    
    x = np.arange(len(pivot_multimodal.index))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, pivot_multimodal['Text-Only'], width, 
                   label='Image Optional Questions', color=COLORS['unimodal'], 
                   alpha=0.8, edgecolor='white', linewidth=1.5)
    bars2 = ax.bar(x + width/2, pivot_multimodal['Multimodal'], width,
                   label='Image Required Questions', color=COLORS['multimodal'], 
                   alpha=0.8, edgecolor='white', linewidth=1.5)
    
    # Add value labels on bars with sample sizes
    def add_value_labels(bars, values, samples, offset=0.01):
        for bar, val, n in zip(bars, values, samples):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + offset,
                   f'{val:.3f}\n(n={n})', ha='center', va='bottom', 
                   fontweight='bold', fontsize=10)
    
    add_value_labels(bars1, pivot_multimodal['Text-Only'], sample_pivot['Text-Only'])
    add_value_labels(bars2, pivot_multimodal['Multimodal'], sample_pivot['Multimodal'])
    
    ax.set_xlabel('Model', fontweight='bold')
    ax.set_ylabel('Accuracy', fontweight='bold')
    # ax.set_title('Performance Comparison: Image Required vs Image Optional Questions', fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(pivot_multimodal.index, rotation=0)
    ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=True)
    ax.set_ylim(0, max(pivot_multimodal.max()) * 1.15)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, format='pdf', bbox_inches='tight', dpi=300,
                   facecolor='white', edgecolor='none')
        print(f"✓ Multimodal comparison saved to {save_path}")
    
    plt.show()
    
    # Enhanced statistics output (keeping the same detailed analysis)
    print("\n" + "="*80)
    print("📊 MULTIMODAL VS TEXT-ONLY PERFORMANCE ANALYSIS")
    print("="*80)
    
    # Create detailed table
    detailed_results = pivot_multimodal.copy()
    detailed_results['Difference (Multi - Text)'] = (detailed_results['Multimodal'] - 
                                                    detailed_results['Text-Only'])
    detailed_results['Relative Change (%)'] = ((detailed_results['Difference (Multi - Text)'] / 
                                               detailed_results['Text-Only']) * 100)
    
    print("\nDetailed Performance Metrics:")
    print("-" * 80)
    for col in detailed_results.columns:
        if col in ['Text-Only', 'Multimodal']:
            detailed_results[col] = detailed_results[col].apply(lambda x: f"{x:.3f}")
        else:
            detailed_results[col] = detailed_results[col].apply(lambda x: f"{x:+.3f}")
    
    print(detailed_results.to_string())
    
    # Summary statistics
    print(f"\n📈 Summary Statistics:")
    print(f"   • Average Text-Only Performance: {pivot_multimodal['Text-Only'].mean():.3f}")
    print(f"   • Average Multimodal Performance: {pivot_multimodal['Multimodal'].mean():.3f}")
    print(f"   • Overall Performance Gap: {(pivot_multimodal['Multimodal'] - pivot_multimodal['Text-Only']).mean():+.3f}")
    
    better_multi = sum((pivot_multimodal['Multimodal'] - pivot_multimodal['Text-Only']) > 0)
    total_models = len(pivot_multimodal)
    print(f"   • Models performing better on multimodal: {better_multi}/{total_models} ({better_multi/total_models*100:.1f}%)")
    
    return pivot_multimodal

def analyze_2025_vs_previous_years(all_models_data):
    """Analyze performance on 2025 questions vs average of previous years with enhanced formatting"""
    
    results_table = []
    
    print("\n" + "="*100)
    print("📅 2025 vs PREVIOUS YEARS PERFORMANCE COMPARISON")
    print("="*100)
    
    for model_name, data in all_models_data.items():
        df = data['df']
        
        # Separate 2025 and previous years
        df_2025 = df[df['year'] == 2025]
        df_previous = df[df['year'] < 2025]
        
        if len(df_2025) > 0 and len(df_previous) > 0:
            # Calculate accuracies
            accuracy_2025 = df_2025['is_correct'].mean()
            accuracy_previous = df_previous['is_correct'].mean()
            
            # Count questions
            count_2025 = len(df_2025)
            count_previous = len(df_previous)
            
            # Calculate difference
            difference = accuracy_2025 - accuracy_previous
            
            # Get year range for previous years
            previous_years = sorted(df_previous['year'].unique())
            year_range = f"{min(previous_years)}-{max(previous_years)}"
            
            # Calculate statistical significance (basic test)
            from scipy.stats import chi2_contingency
            
            # Create contingency table
            contingency = np.array([
                [df_previous['is_correct'].sum(), len(df_previous) - df_previous['is_correct'].sum()],
                [df_2025['is_correct'].sum(), len(df_2025) - df_2025['is_correct'].sum()]
            ])
            
            try:
                chi2, p_value, _, _ = chi2_contingency(contingency)
                significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
            except:
                significance = ""
                p_value = np.nan
            
            results_table.append({
                'Model': model_name,
                'Years (Previous)': year_range,
                'Previous Accuracy': f"{accuracy_previous:.3f}",
                'Previous Count': count_previous,
                '2025 Accuracy': f"{accuracy_2025:.3f}",
                '2025 Count': count_2025,
                'Difference': f"{difference:+.3f}{significance}",
                'Relative Change (%)': f"{(difference / accuracy_previous) * 100:+.1f}%",
                'p-value': f"{p_value:.4f}" if not np.isnan(p_value) else "N/A"
            })
            
        else:
            print(f"⚠️  Warning: {model_name} doesn't have data for both 2025 and previous years")
    
    # Create and display enhanced DataFrame
    if results_table:
        results_df = pd.DataFrame(results_table)
        
        print("\n📊 Comprehensive Performance Comparison:")
        print("-" * 100)
        
        # Create a formatted table string
        col_widths = {
            'Model': 12,
            'Years (Previous)': 10,
            'Previous Accuracy': 12,
            'Previous Count': 8,
            '2025 Accuracy': 12,
            '2025 Count': 8,
            'Difference': 12,
            'Relative Change (%)': 15,
            'p-value': 8
        }
        
        # Print header
        header = "|".join([col.center(col_widths[col]) for col in results_df.columns])
        print(header)
        print("-" * len(header))
        
        # Print rows
        for _, row in results_df.iterrows():
            formatted_row = "|".join([
                str(row[col]).center(col_widths[col]) for col in results_df.columns
            ])
            print(formatted_row)
        
        print("-" * len(header))
        
        # Enhanced summary statistics
        print(f"\n📈 Key Insights:")
        
        # Convert back to numeric for calculations
        numeric_diffs = [float(row['Difference'].replace('*', '').replace('+', '')) 
                        for _, row in results_df.iterrows()]
        numeric_changes = [float(row['Relative Change (%)'].replace('%', '').replace('+', '')) 
                          for _, row in results_df.iterrows()]
        
        avg_diff = np.mean(numeric_diffs)
        avg_change = np.mean(numeric_changes)
        
        better_count = sum(1 for diff in numeric_diffs if diff > 0)
        worse_count = sum(1 for diff in numeric_diffs if diff < 0)
        total_count = len(numeric_diffs)
        
        print(f"   • Average performance difference: {avg_diff:+.3f}")
        print(f"   • Average relative change: {avg_change:+.1f}%")
        print(f"   • Models performing better on 2025: {better_count}/{total_count} ({better_count/total_count*100:.1f}%)")
        print(f"   • Models performing worse on 2025: {worse_count}/{total_count} ({worse_count/total_count*100:.1f}%)")
        
        if any('*' in str(row['Difference']) for _, row in results_df.iterrows()):
            print(f"\n📋 Statistical Significance: * p<0.05, ** p<0.01, *** p<0.001")
        
        return pd.DataFrame([{
            'Model': row['Model'],
            'Previous_Years_Range': row['Years (Previous)'],
            'Previous_Years_Accuracy': float(row['Previous Accuracy']),
            'Previous_Years_Count': row['Previous Count'],
            '2025_Accuracy': float(row['2025 Accuracy']),
            '2025_Count': row['2025 Count'],
            'Difference': float(row['Difference'].replace('*', '').replace('+', '')),
            'Relative_Change_Percent': float(row['Relative Change (%)'].replace('%', '').replace('+', '')),
            'p_value': float(row['p-value']) if row['p-value'] != 'N/A' else np.nan
        } for _, row in results_df.iterrows()])
    
    return pd.DataFrame()

def run_multi_model_analysis(model_folders, dataset_path, save_plots=True):
    """Run the complete multi-model analysis with enhanced output"""
    
    print("🚀 " + "="*80)
    print("   MULTI-MODEL PERFORMANCE ANALYSIS PIPELINE")
    print("="*85)
    print(f"📁 Models to analyze: {', '.join(model_folders.keys())}")
    print(f"📊 Original dataset: {os.path.basename(dataset_path)}")
    print(f"💾 Save plots as PDF: {'Yes' if save_plots else 'No'}")
    print("-"*85)
    
    # Load data from all models
    print("\n📥 Loading evaluation data...")
    all_models_data = load_multi_model_data(model_folders, dataset_path)
    
    if not all_models_data:
        print("❌ No data loaded. Please check your folder paths.")
        return
    
    # Print data summary
    print(f"\n✅ Successfully loaded data for {len(all_models_data)} models:")
    total_questions = 0
    for model_name, data in all_models_data.items():
        n_questions = len(data['df'])
        total_questions += n_questions
        print(f"   • {model_name}: {n_questions:,} questions")
    print(f"   📊 Total questions across all models: {total_questions:,}")
    
    # 1. Create accuracy heatmap by model
    print("\n" + "="*50)
    print("📈 1. GENERATING ACCURACY HEATMAPS")
    print("="*50)
    heatmap_path = "model_accuracy_heatmap.pdf" if save_plots else None
    pivot_subject, pivot_language = create_accuracy_heatmap_by_model(all_models_data, heatmap_path)
    
    # 2. Create multimodal vs unimodal comparison
    print("\n" + "="*50)
    print("🖼️  2. ANALYZING MULTIMODAL PERFORMANCE")
    print("="*50)
    multimodal_path = "multimodal_comparison.pdf" if save_plots else None
    multimodal_results = create_multimodal_comparison(all_models_data, multimodal_path)
    
    # 3. Analyze 2025 vs previous years
    print("\n" + "="*50)
    print("📅 3. TEMPORAL PERFORMANCE ANALYSIS")
    print("="*50)
    year_comparison = analyze_2025_vs_previous_years(all_models_data)
    
    # Final summary
    print("\n" + "🎯 " + "="*80)
    print("   ANALYSIS COMPLETE - SUMMARY")
    print("="*85)
    
    if save_plots:
        print("📄 Generated Files:")
        print("   • model_accuracy_heatmap.pdf - Subject & Language performance heatmaps")
        print("   • multimodal_comparison.pdf - Multimodal vs Text-only comparison")
    
    print("\n📊 Key Findings:")
    
    # Overall best performing model
    combined_df = pd.concat([data['df'] for data in all_models_data.values()], ignore_index=True)
    overall_performance = combined_df.groupby('model')['is_correct'].mean().sort_values(ascending=False)
    best_model = overall_performance.index[0]
    best_accuracy = overall_performance.iloc[0]
    
    print(f"   🏆 Best overall model: {best_model} ({best_accuracy:.3f} accuracy)")
    
    # Multimodal performance leader
    if 'Multimodal' in multimodal_results.columns:
        multi_leader = multimodal_results['Multimodal'].idxmax()
        multi_score = multimodal_results.loc[multi_leader, 'Multimodal']
        print(f"   🖼️  Best multimodal performance: {multi_leader} ({multi_score:.3f} accuracy)")
    
    # 2025 performance
    if not year_comparison.empty and '2025_Accuracy' in year_comparison.columns:
        best_2025 = year_comparison.loc[year_comparison['2025_Accuracy'].idxmax(), 'Model']
        best_2025_score = year_comparison['2025_Accuracy'].max()
        print(f"   📅 Best 2025 performance: {best_2025} ({best_2025_score:.3f} accuracy)")
    
    print("\n✨ Analysis pipeline completed successfully!")
    print("="*85)
    
    return {
        'model_data': all_models_data,
        'accuracy_by_subject': pivot_subject,
        'accuracy_by_language': pivot_language,
        'multimodal_comparison': multimodal_results,
        'year_comparison': year_comparison,
        'overall_performance': overall_performance
    }


model_folders = {
    "Gemma 3  27B": "gemma3_evaluation_results",
    "Gemini 2.5 Pro": "gemini25_evaluation_results",
    "InternVL3 8B": "internvl3_8b_evaluation_results",
    "Qwen 2.5 VL 7B": "qwen25_evaluation_results"
}

# Path to the original dataset
dataset_path = "jee_advanced_combined_fixed.csv"

# Run the analysis with publication-quality outputs
print("🎯 Starting publication-ready analysis...")
results = run_multi_model_analysis(model_folders, dataset_path, save_plots=True)

if results:
    print("\n🎉 Success!")
else:
    print("❌ Analysis failed. Please check your file paths and data.")